In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import actipy
from scipy.interpolate import interp1d
from scipy import constants
from datetime import datetime
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

# Thesis sample plots

In [ ]:
def parse_datetime_from_timestamp(time_in_secs):
    return datetime.fromtimestamp(float(time_in_secs))

def plot_window(data: pd.DataFrame, window=[], title="", legend=[], sample_hz=100,                 
                resample_hz=0, period=10, random_state=42, ylabel="Acceleration [g]",
                axes=None, save_path=None):
    np.random.seed(random_state)
    
    if not window:
        start_index = np.random.randint(0, len(data) - period*sample_hz)
        window = [start_index, start_index + period*sample_hz]
    
    plot_data = data[['x', 'y', 'z']].iloc[window[0]:window[1]]
    
    if resample_hz and resample_hz != sample_hz:
        plot_data = resize(plot_data, period, resample_hz, axis=0)

    plot_data.index = (plot_data.index - plot_data.index[0])/pd.Timedelta(seconds=1)

    if axes is None:
        fig, axes = plt.subplots(figsize=(8, 4), dpi=1000)
    
    else:
        fig = plt.gcf()

    axes.plot(plot_data)
    axes.set_facecolor('none')


    if title:
        axes.set_title(title)
    
    if legend:
        axes.legend(legend)
    
    axes.set_xlabel("Elapsed time [s]")
    axes.set_ylabel(ylabel)

    if save_path:
        fig.savefig(save_path, transparent=True)

def resize(x, period, resample_hz, axis=1):
    length =  int(resample_hz * period)
    length_orig = x.shape[axis]
    t_orig = np.linspace(0, 1, length_orig, endpoint=True)
    t_new = np.linspace(0, 1, length, endpoint=True)
    x_new = interp1d(t_orig, x, kind="linear", axis=axis, assume_sorted=True)(t_new)
    
    resampled_data = pd.DataFrame(x_new, columns=x.columns)
    resampled_data.index = pd.TimedeltaIndex(t_new * period, unit='s')
    
    return resampled_data

def preprocess_data(data, sample_rate):
    data, _ = actipy.process(data, sample_rate, verbose=False)

    return data

1. raw oxwalk accelerometer data

In [ ]:
oxwalk_raw = pd.read_csv("data/OxWalk_Dec2022/Wrist_100Hz/P04_wrist100.csv",
                         index_col="timestamp", parse_dates=True)

2. Processed oxwalk

In [ ]:
oxwalk_proc = preprocess_data(oxwalk_raw, 100)

3. raw ldopa

In [ ]:
ldopa_raw = pd.read_csv("data/LDOPA_DATA/GENEActiv/patient4/rawdata_day4.txt",
                        delimiter="\t", index_col="timestamp",
                        parse_dates=True, skipinitialspace=True,
                        date_parser=parse_datetime_from_timestamp)

ldopa_raw = ldopa_raw.rename(columns={"GENEActiv_X": "x", 
                                      "GENEActiv_Y": "y", 
                                      "GENEActiv_Z": "z"})[["x", "y", "z"]]

4. processed ldopa

In [ ]:
ldopa_proc = preprocess_data(ldopa_raw/constants.g, 50)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 6), dpi=800)

plot_window(oxwalk_raw,  title="Raw AX3 Axivity data", sample_hz=100, 
            random_state=50, axes=axs[0, 0])
plot_window(oxwalk_proc, title="Prepared AX3 Axixity data", sample_hz=100, 
            resample_hz=30, random_state=50, axes=axs[1, 0])
plot_window(ldopa_raw, title="Raw GENEActiv data", sample_hz=50, axes=axs[0, 1],
            ylabel="Acceleration [m/s²]")
plot_window(ldopa_proc, title="Prepared GENEActiv data", sample_hz=50, 
            resample_hz=30, axes=axs[1, 1])

fig.legend(["x", "y", "z"])

plt.tight_layout()
plt.show()

In [ ]:
plot_window(oxwalk_proc, sample_hz=100,
            resample_hz=30, random_state=10, period=30, save_path="outputs/plots/samples.png")

In [ ]:
# Simulate confusion matrix
np.random.seed(452)
labels_true = np.zeros(360, dtype=int)
labels_true[:36] = 1  # 10% walking
np.random.shuffle(labels_true)

labels_pred = np.random.choice([0, 1], size=360, p=[0.95, 0.05])
cm = confusion_matrix(labels_true, labels_pred)
accuracy = accuracy_score(labels_true, labels_pred)
f1 = f1_score(labels_true, labels_pred)

plt.figure(figsize=(8, 6), dpi=1000)
plt.rcParams.update({'font.size': 14}) 
sns.heatmap(cm, annot=True, fmt='d', cmap='viridis', annot_kws={'size': 40})

plt.title(f'Confusion Matrix\nAccuracy: {accuracy:.2f}, F1 Score: {f1:.2f}')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks([0.5, 1.5], ['Not Walking', 'Walking'])
plt.yticks([0.5, 1.5], ['Not Walking', 'Walking'])
plt.show()

# Old sample plots

In [ ]:
X = np.load('prepared_data/both/X.npy')
Y = np.load('prepared_data/both/Y.npy')
day = np.load('prepared_data/both/day.npy')
source = np.load('prepared_data/both/S.npy')
pid = np.load('prepared_data/both/P.npy')
Y_pred_both_trained_all = np.load('outputs/predictions/y_pred_ssl_train_all_test_all.npy')
Y_pred_oxwalk_trained = np.load('outputs/predictions/y_pred_ssl_train_OXWALK_test_LDOPA.npy')
Y_pred_ldopa_trained = np.load('outputs/predictions/y_pred_ssl_train_LDOPA_test_LDOPA.npy')

In [ ]:
Y_pred_both_trained_all_rf = np.load('outputs/predictions/y_pred_rf_train_all_test_all.npy')
Y_pred_oxwalk_trained_rf = np.load('outputs/predictions/y_pred_rf_train_OXWALK_test_LDOPA.npy')
Y_pred_ldopa_trained_rf = np.load('outputs/predictions/y_pred_rf_train_LDOPA_test_LDOPA.npy')

In [ ]:
oxwalk_mask = source == "OXWALK"

X_oxwalk = X[oxwalk_mask]
Y_oxwalk = Y[oxwalk_mask]
pid_oxwalk = pid[oxwalk_mask]

In [ ]:
ldopa_mask = source == "LDOPA"
X_ldopa = X[ldopa_mask]
Y_ldopa = Y[ldopa_mask]
pid_ldopa = pid[ldopa_mask]
day_ldopa = day[ldopa_mask]
Y_pred_both_trained = Y_pred_both_trained_all[ldopa_mask]
Y_pred_both_trained_rf = Y_pred_both_trained_all_rf[ldopa_mask]

In [ ]:
def plot_samples(X, Y, main_title="Accelerometry Traces"):
    NPLOTS = 8
    Y = np.array([{'not-walking': 'Not Walking', 'walking': 'Walking'}[elem] for elem in Y])
    unqY = np.unique(Y)
    np.random.seed(42)
    fig, axs = plt.subplots(NPLOTS, len(unqY), sharex=True, sharey=True, figsize=(12, NPLOTS*2))
    fig.suptitle(main_title, fontsize=16)

    for y, col in zip(unqY, axs.T):
        idxs = np.random.choice(np.where(Y==y)[0], size=NPLOTS)
        col[0].set_title(y)
        for x, ax in zip(X[idxs], col):
            time_points = np.arange(0, len(x)) / 30
            ax.plot(time_points, np.linalg.norm(x, axis=1) - 1)
            ax.set_ylim(-2, 2)
            ax.set_xlim(0, len(x) / 30) 
            ax.set_xlabel('Time (s)')

    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
plot_samples(X_oxwalk, Y_oxwalk, "Samples taken from healthy subjects")

In [ ]:
plot_samples(X_ldopa[day_ldopa == 1], Y_ldopa[day_ldopa == 1], "Samples taken from PD subjects on medication")

In [ ]:
plot_samples(X_ldopa[day_ldopa == 4], Y_ldopa[day_ldopa == 4], "Samples taken from PD subjects off medication")